# Titanic 생존
`titanic.ipynb`

## 참고할 제언
- imputer 사용  > age 결측치 채우고 , embarked 수동으로 결측치 세우기를 함. 
- OHE 원핫인코딩 > 범주형 데이터를 여러개의 컬럼으로 변환 
- Pros -> 숫자 0, 1, 2 는 크기 순서가 생겨서 해석이 잘못될 수 있음
- Cons -> 카테고리가 많으면 차원이 늘어남 
- MICE: 결측치가 있는 변수를 다른 변수를 이용해 반복적으로 예측한다. 


In [1]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
# 실험실에서 기능 활성화
from sklearn.experimental import enable_iterative_imputer
# MICE (다른 특성으로 결측 대체하기)
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

titanic = sns.load_dataset('titanic')

features = ['pclass', 'sex', 'age', 'fare', 'embarked']
target = 'survived'

df = titanic[features + [target]]

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    891 non-null    int64  
 1   sex       891 non-null    object 
 2   age       714 non-null    float64
 3   fare      891 non-null    float64
 4   embarked  889 non-null    object 
 5   survived  891 non-null    int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 41.9+ KB


In [ ]:
# knn(knc)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled,y_test)

